# Introduction to Regular Expression (RegEx)

Documentation link : [RE Link](https://docs.python.org/3/library/re.html#),
                     [Regular Expression How To](https://docs.python.org/3/howto/regex.html#regex-howto)

In [2]:
import re
import random

Regular expressions (called REs, or regexes, or regex patterns) are essentially a tiny, highly specialized programming language embedded inside Python and made available through the re module. 

Using this little language, you specify the rules for the set of possible strings that you want to match; this set might contain English sentences, or e-mail addresses, or TeX commands, or anything you like. You can then ask questions such as **“Does this string match the pattern?”**, or **“Is there a match for the pattern anywhere in this string?”**. You can also use REs to modify a string or to split it apart in various ways.

The regular expression language is relatively small and restricted, so not all possible string processing tasks can be done using regular expressions. There are also tasks that can be done with regular expressions, but the expressions turn out to be very complicated. In these cases, you may be better off writing Python code to do the processing; while Python code will be slower than an elaborate regular expression, it will also probably be more understandable.

### Simple Patterns - Matching Characters

Most letters and characters will simply match themselves. For example, the regular expression `test` will match the string `test` exactly. (You can enable a case-insensitive mode that would let this RE match `Test` or `TEST` as well; more about this later.)

#### Compiling Regular Expressions

Regular expressions are compiled into pattern objects, which have methods for various operations such as searching for pattern matches or performing string substitutions.

In [38]:
pattern = re.compile('ab*')
pattern

re.compile(r'ab*', re.UNICODE)

`re.compile()` also accepts an optional flags argument, used to enable various special features and syntax variations. We’ll go over the available settings later, but for now a single example will do:

In [40]:
pattern = re.compile('ab*', re.IGNORECASE)
pattern

re.compile(r'ab*', re.IGNORECASE|re.UNICODE)

The RE is passed to `re.compile()` as a string. REs are handled as strings because regular expressions aren’t part of the core Python language, and no special syntax was created for expressing them. Instead, the re module is simply a C extension module included with Python, just like the socket or zlib modules.

Putting REs in strings keeps the Python language simpler, but has one disadvantage which is called `The Backslash Plague`

#### The Backslash Plague

Regular expressions use the backslash character (`'\'`) to indicate special forms or to allow special characters to be used without invoking their special meaning. This conflicts with Python’s usage of the same character for the same purpose in string literals.

Let’s say you want to write a RE that matches the string `\section` from the document.

* Start with the desired string to be matched. 
* Escape any backslashes and other metacharacters by preceding them with a backslash
* Resulting in the string `\\section`. 

The resulting string that must be passed to `re.compile()` must be \\section. However, to express this as a Python string literal, both backslashes must be escaped again.

| Characters | Remarks | 
| :---: | :--- |
| `\section` | Text string to be matched |
| `\\section` | Escaped backslasges for `re.compile()`|
| `\\\\section` | Escaped backslaces for a string literal|

In short, to match a literal backslash, one has to write '\\\\' as the RE string, because the regular expression must be \\, and each backslash must be expressed as \\ inside a regular Python string literal. In REs that feature backslashes repeatedly, this leads to lots of repeated backslashes and makes the resulting strings difficult to understand.

The solution is to use Python’s **raw string notation for regular expressions;** backslashes are not handled in any special way in a string literal prefixed with 'r', so r"\n" is a two-character string containing '\' and 'n', while "\n" is a one-character string containing a newline. Regular expressions will often be written in Python code using this raw string notation.

In addition, special escape sequences that are valid in regular expressions, but not valid as Python string literals, now result in a DeprecationWarning and will eventually become a SyntaxError, which means the sequences will be invalid if raw string notation or escaping the backslashes isn’t used.

| Regular String | Raw Strings | 
| :---: | :--- |
| `ab*` | r"ab*" |
| `\\\\section` | r"\\section"|
| `\\w+\\s+\\1` | r"\w+\s+\1"|

In [15]:
# finditer method all occurance of the pattern in the string

test_string = '123abc456789abc123ABC'
pattern = re.compile(r"abc")
matches = pattern.finditer(test_string)
print("Oject Type of 'matches':",type(matches))

for match in matches:
    print(match)

Oject Type of 'matches': <class 'callable_iterator'>
<re.Match object; span=(3, 6), match='abc'>
<re.Match object; span=(12, 15), match='abc'>


In [175]:
# Findall method find all occurances of the pattern in the string
test_string = '123abc456789abc123ABC'
pattern = re.compile(r"abc")
matches = pattern.findall(test_string)
matches

['abc', 'abc']

There are exceptions to this rule; some characters are special metacharacters, and don’t match themselves. Instead, they signal that some out-of-the-ordinary thing should be matched, or they affect other portions of the RE by repeating them or changing their meaning. Much of this document is devoted to discussing various metacharacters and what they do.

Here’s a complete list of the metacharacters 

### Meta Characters
* `.`  - Any Character Except New Line
* `^`  - Start with "^hello"
*` $`  - End with "world$"
* `*`  - Zero or more occurances "aix*"
* `+`  - One or more occurances "aix+"
* `{ }` - Exactly the specified number of occurances al{2}
* `[ ]` - A set of characters [a-m]
* `\`   - Special sequance or escape special characters "\$"
* `|`   - Either or "abc|ABC"
* `( )` - Capture and group

#### First Meta Character - [ ] 

They’re used for specifying a character class, which is a set of characters that you wish to match. Characters can be listed individually, or a range of characters can be indicated by giving two characters and separating them by a '-'. For example, [abc] will match any of the characters a, b, or c; this is the same as [a-c], which uses a range to express the same set of characters. If you wanted to match only lowercase letters, your RE would be [a-z]

In [132]:
# Example
# It Looks for characters a,b 
# It looks for characters in the range of 7 to 9 and f to h

test_string = '123abc456789abcdefgh123ABC'
pattern = re.compile(r"[ab7-9f-h]")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(3, 4), match='a'>
<re.Match object; span=(4, 5), match='b'>
<re.Match object; span=(9, 10), match='7'>
<re.Match object; span=(10, 11), match='8'>
<re.Match object; span=(11, 12), match='9'>
<re.Match object; span=(12, 13), match='a'>
<re.Match object; span=(13, 14), match='b'>
<re.Match object; span=(17, 18), match='f'>
<re.Match object; span=(18, 19), match='g'>
<re.Match object; span=(19, 20), match='h'>


Metacharacters are not active inside classes. For example, `[akm$]` will match any of the characters 'a', 'k', 'm', or '$'; '$' is usually a metacharacter, but inside a character class it’s stripped of its special nature

In [133]:
# Example
# It Looks for characters a,k,m and $ 
# $ is a meta character, but do not have the special nature inside the class

test_string = 'abckgh$1'
pattern = re.compile(r"[akm$]")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(3, 4), match='k'>
<re.Match object; span=(6, 7), match='$'>


You can match the characters not listed within the class by complementing the set. This is indicated by including a '^' as the first character of the class. For example, `[^5]` will match any character except '5'. If the caret appears elsewhere in a character class, it does not have special meaning. For example: `[5^]` will match either a '5' or a '^'

In [43]:
# Example ^ in the starting position
# It Looks for characters not a,k,m and $ 
# $ is a meta character, but do not have the special nature inside the class

test_string = 'akgh$1'
pattern = re.compile(r"[^akm$]")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(2, 3), match='g'>
<re.Match object; span=(3, 4), match='h'>
<re.Match object; span=(5, 6), match='1'>


In [44]:
# Example ^ not in the starting position
# It Looks for characters not a,k,^,m and $ 
# ^ and $ are meta characters, but do not have the special nature inside the class

test_string = 'ak^gh$1'
pattern = re.compile(r"[ak^m$]")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(1, 2), match='k'>
<re.Match object; span=(2, 3), match='^'>
<re.Match object; span=(5, 6), match='$'>


Perhaps the most important metacharacter is the backslash, `\`. As in Python string literals, the backslash can be followed by various characters to signal various special sequences. It’s also used to escape all the metacharacters so you can still match them in patterns; for example, if you need to match a `[` or `\`, you can precede them with a backslash to remove their special meaning: `\[` or `\\`.

Some of the special sequences beginning with '\' represent predefined sets of characters that are often useful, such as the set of digits, the set of letters, or the set of anything that isn’t whitespace.

Let’s take an example: `\w` matches any alphanumeric character. If the regex pattern is expressed in bytes, this is equivalent to the class [a-zA-Z0-9_]. If the regex pattern is a string, `\w` will match all the characters marked as letters in the Unicode database provided by the unicodedata module. You can use the more restricted definition of `\w` in a string pattern by supplying the `re.ASCII` flag when compiling the regular expression.

The following list of special sequences isn’t complete. For a complete list of sequences and expanded class definitions for Unicode string patterns, see the last part of Regular Expression Syntax in the Standard Library reference. In general, the Unicode versions match any character that’s in the appropriate category in the Unicode database.

#### More Meta Characters
* `\d` - Matches any decimal digits that is equivalent to the class `[0-9]`
* `\D` - Matches any non-decimal digit characters, that is equivalent to the class `[^0-9]`
* `\s` - Matches any Whitespace (space, tab, newline)`[\t\n\r\f\v]`
* `\S` - Matches any non-Whitespace `[^\t\n\r\f\v]`
* `\w` - Matches any alpha-numeric (Word) Character (a-z, A-Z, 0-9, _) `[a-zA-Z0-9_]`
* `\W` - Matches any non-alpha-numeric (Word) Character `[^a-zA-Z0-9_]`
* `\b` - Matches where the specified characters are at the begining or at the end of word (Word Boundary)
* `\B` - Matches where the specified characters are at the begining or at the end of word (Not a Word Boundary)

These sequences can be included inside a character class. For example, `[\s,.]` is a character class that will match any whitespace character, or ',' or '.'.

The final metacharacter in this section is `.` 

It matches anything except a newline character, and there’s an alternate mode (re.DOTALL) where it will match even a newline. `.` is often used where you want to match “any character”.

### Repeating Things

Being able to match varying sets of characters is the first thing regular expressions can do that isn’t already possible with the methods available on strings. However, if that was the only additional capability of regexes, they wouldn’t be much of an advance. Another capability is that you can specify that portions of the RE must be repeated a certain number of times.

#### Quantifiers
* `*` - 0 or More
* `+` - 1 or more
* `?` - 0 or 1 -> Optional Character
* `{4}` - Exact Number of elements to get after the match
* `{4,6}` - Range number (min, max)

The first metacharacter for repeating things that we’ll look at is `*`. `*` doesn’t match the literal character '*'; instead, it specifies that the previous character can be matched zero or more times, instead of exactly once.

For example, ca*t will match 'ct' (0 'a' characters), 'cat' (1 'a'), 'caaat' (3 'a' characters), and so forth.

In [46]:
# Example for Quantifers - * 

test_string = 'ctandabcathdhcaaatandmore'
pattern = re.compile(r"ca*t")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 2), match='ct'>
<re.Match object; span=(7, 10), match='cat'>
<re.Match object; span=(13, 18), match='caaat'>


Repetitions such as `*` are greedy; when repeating a RE, the matching engine will try to repeat it as many times as possible. If later portions of the pattern don’t match, the matching engine will then back up and try again with fewer repetitions.

A step-by-step example will make this more obvious. Let’s consider the expression `a[bcd]*b`. This matches the letter 'a', zero or more letters from the class `[bcd]`, and finally ends with a 'b'. Now imagine matching this RE against the string 'abcbd'.


In [92]:
# Example for Quantifers - * 

test_string = 'abcbdbacddb'
pattern = re.compile(r"a[bcd]*b")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 6), match='abcbdb'>
<re.Match object; span=(6, 11), match='acddb'>


In [86]:
test_string = 'abcdefghijklmnopabddkshdaskjdhsakdacdqweuqwioeuqwoeuadjkgdkasdksd'
pattern = re.compile(r"a[bc]*d")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 4), match='abcd'>
<re.Match object; span=(16, 19), match='abd'>
<re.Match object; span=(34, 37), match='acd'>
<re.Match object; span=(52, 54), match='ad'>


The Search for the following combinations
1. a bcd b
2. a bc  b
3. a bd  b
4. a cd  b

From the string of `abcbd`
1. abcb is the only combination you can get.

| Step | Matched | Explanation |
| :---: | :---: | :--- |
| Step 1 | a | The `a` in the RE matches | 
| Step 2 | abcbd | The engine matches `[bcd]*`, going as fars as it can, which is to the end of the string. | 
| Step 3 | Failure | The engine tries to match `b`, but the current position is at the end of the string, so it fails. | 
| Step 4 | abcb | Back up, so that `[bcd]*` matches one less character. | 
| Step 5 | Failure | Try `b` again, but the current position is at the last character, which is a `d`. | 
| Step 6 | abc | Back up again, so that `[bcd]*` is only matching `bc`. | 
| Step 6 | abcb | Try `b` again. This time the character at the current position is `b`, so it succeeds. | 

The end of the RE has now been reached, and it has matched 'abcb'. This demonstrates how the matching engine goes as far as it can at first, and if no match is found it will then progressively back up and retry the rest of the RE again and again. It will back up until it has tried zero matches for [bcd]*, and if that subsequently fails, the engine will conclude that the string doesn’t match the RE at all.

Another repeating metacharacter is +, which matches one or more times. Pay careful attention to the difference between * and +; * matches zero or more times, so whatever’s being repeated may not be present at all, while + requires at least one occurrence. To use a similar example, ca+t will match 'cat' (1 'a'), 'caaat' (3 'a's), but won’t match 'ct'.

In [93]:
# Example for Quantifers - +

test_string = 'ctandabcathdhcaaatandmore'
pattern = re.compile(r"ca+t")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(7, 10), match='cat'>
<re.Match object; span=(13, 18), match='caaat'>


There are two more repeating qualifiers. The question mark character, ?, matches either once or zero times; you can think of it as marking something as being optional. For example, home-?brew matches either 'homebrew' or 'home-brew'.

In [139]:
# Example for Quantifers - ?

test_string = 'Today Infosys golive with SFSF, the Go-Live was very successful.'
pattern = re.compile(r"go-?live", re.I)
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(14, 20), match='golive'>
<re.Match object; span=(36, 43), match='Go-Live'>


The most complicated repeated qualifier is {m,n}, where m and n are decimal integers. This qualifier means there must be at least m repetitions, and at most n. For example, ca{2,3}t will match caat and caaat. It won’t match 'cat'.

You can omit either m or n; in that case, a reasonable value is assumed for the missing value. Omitting m is interpreted as a lower limit of 0, while omitting n results in an upper bound of infinity.

Readers of a reductionist bent may notice that the three other qualifiers can all be expressed using this notation. {0,} is the same as *, {1,} is equivalent to +, and {0,1} is the same as ?. It’s better to use *, +, or ? when you can, simply because they’re shorter and easier to read

In [140]:
# Example for Quantifers - { }
# Having a hard limit of 2 repeatations

test_string = 'ctxcatcaatcaaaty'
pattern = re.compile(r"ca{2}t")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(6, 10), match='caat'>


In [141]:
# Example for Quantifers - { }
# Having a range limit of 2 to 3 repeatations
test_string = 'ctxcatcaatcaaaty'
pattern = re.compile(r"ca{2,3}t")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(6, 10), match='caat'>
<re.Match object; span=(10, 15), match='caaat'>


In [142]:
# Example for Quantifers - { }
# Having a start limit undefined, hence range starts from 0
# Same as ca*t

test_string = 'ctxcatcaatcaaaty'
pattern = re.compile(r"ca{,3}t")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 2), match='ct'>
<re.Match object; span=(3, 6), match='cat'>
<re.Match object; span=(6, 10), match='caat'>
<re.Match object; span=(10, 15), match='caaat'>


***

### Performing Matches

Once you have an object representing a compiled regular expression, what do you do with it? Pattern objects have several methods and attributes. Only the most significant ones will be covered here; consult the re docs for a complete listing.

| Method / Attribute | Purpose |
| :--- | :--- |
| match() | Determine if the RE matches at the beginning of the string. |
| search() | Scan through a string, looking for any location where this RE matches. |
| findall() | Find all substrings where the RE matches, and returns them as a list. |
| finditer() | Find all substrings where the RE matches, and returns them as an iterator. |


`match()` and `search()` return `None` if no match can be found. If they’re successful, a match object instance is returned, containing information about the match: where it starts and ends, the substring it matched, and more.

In [102]:
test_string = '123abc456789abc123ABC'
pattern = re.compile(r'[1-9]+')
pattern

re.compile(r'[1-9]+', re.UNICODE)

Now, you can try matching various strings against the RE `[a-z]+`. An empty string shouldn’t match at all, since + means ‘one or more repetitions’. `match()` should return `None` in this case, which will cause the interpreter to print no output. You can explicitly print the result of `match()` to make this clear.

In [95]:
print(pattern.match(""))

None


Now, let’s try it on a string that it should match, such as tempo. In this case, `match()` will return a match object, so you should store the result in a variable for later use.

In [104]:
matches = pattern.match(test_string)
matches

<re.Match object; span=(0, 3), match='123'>

Now you can query the match object for information about the matching string. Match object instances also have several methods and attributes; the most important ones are:

| Method / Attribute | Purpose |
| :--- | :--- |
| group() | Return the string matched by the RE. |
| start() | Return the starting position of the match. |
| end() | Return the ending position of the match. |
| span() | Return a tuple containing the (start, end) positions of the match. |

Trying these methods will soon clarify their meaning

In [158]:
print(matches.group())
print(matches.start())
print(matches.end())
print(matches.span())

123
0
3
(0, 3)


`group()` returns the substring that was matched by the RE. `start()` and `end()` return the starting and ending index of the match. `span()` returns both start and end indexes in a single tuple. Since the `match()` method only checks if the RE matches at the start of a string, `start()` will always be zero. However, the `search()` method of patterns scans through the string, so the match may not start at zero in that case.

In [105]:
pattern = re.compile(r"abc", re.I)
matches = pattern.search(test_string)
print(matches.group())
print(matches.start())
print(matches.end())
print(matches.span())

abc
3
6
(3, 6)


`findall()` has to create the entire list before it can be returned as the result. The `finditer()` method returns a sequence of match object instances as an iterator:

In [106]:
pattern = re.compile(r"abc", re.I)
matches = pattern.findall(test_string)
matches

['abc', 'abc', 'ABC']

In [112]:
pattern = re.compile(r"abc", re.I)
matches = pattern.finditer(test_string)
for match in matches:
    print(match.group())
    print(match.start())
    print(match.end())
    print(match.span())

abc
3
6
(3, 6)
abc
12
15
(12, 15)
ABC
18
21
(18, 21)


***

### Compilation Flags

Compilation flags let you modify some aspects of how regular expressions work. Flags are available in the re module under two names, a long name such as `IGNORECASE` and a short, one-letter form such as `I`. 

Multiple flags can be specified by bitwise OR-ing them; re.I | re.M sets both the I and M flags, for example.

Here’s a table of the available flags, followed by a more detailed explanation of each one.

| Flag | Meaning |
| :--- | :--- |
| ASCII, A | Makes several escapes like \w, \b, \s and \d match only on ASCII characters with the respective property. |
| DOTALL, S | Make . match any character, including newlines. |
| IGNORECASE, I | Do case-insensitive matches. |
| LOCALE, L | Do a locale-aware match. |
| MULTILINE, M | Multi-line matching, affecting ^ and $. |
| VERBOSE, X (for ‘extended’) | Enable verbose REs, which can be organized more cleanly and understandably. |

*** 

### More Patterns - More Meta Characters

zero-width assertions: They don’t cause the engine to advance through the string; instead, they consume no characters at all, and simply succeed or fail. For example, \b is an assertion that the current position is located at a word boundary; the position isn’t changed by the \b at all. This means that zero-width assertions should never be repeated, because if they match once at a given location, they can obviously be matched an infinite number of times.

In [113]:
# Example matches class only when it’s a complete word; 
# It won’t match when it’s contained inside another word.

p = re.compile(r'\bclass\b')

print(p.search('no class at all'))
print(p.search('the declassified algorithm'))
print(p.search('one subclass is'))

<re.Match object; span=(3, 8), match='class'>
None
None


In [169]:
# Example matches class only when it’s a end of the  word or full word
# It won’t match when it’s contained inside another word.

p = re.compile(r'class\b')

print(p.search('no class at all'))
print(p.search('the declassified algorithm'))
print(p.search('one subclass is'))

<re.Match object; span=(3, 8), match='class'>
None
<re.Match object; span=(7, 12), match='class'>


In [170]:
# Example matches class only when it’s in the begining of the  word or full word
# It won’t match when it’s contained inside another word.

p = re.compile(r'\bclass')

print(p.search('no class at all'))
print(p.search('the classified algorithm'))
print(p.search('one subclass is'))

<re.Match object; span=(3, 8), match='class'>
<re.Match object; span=(4, 9), match='class'>
None


2. | Alternation, or the “or” operator: If A and B are regular expressions, A|B will match any string that matches either A or B. `|` has very low precedence in order to make it work reasonably when you’re alternating multi-character strings. `Crow|Servo` will match either 'Crow' or 'Servo', not 'Cro', a 'w' or an 'S', and 'ervo'. To match a literal '|', use \|, or enclose it inside a character class, as in [|].

3. ^ Matches at the beginning of lines: Unless the MULTILINE flag has been set, this will only match at the beginning of the string. In MULTILINE mode, this also matches immediately after each newline within the string. For example, if you wish to match the word From only at the beginning of a line, the RE to use is ^From.

4. $ Matches at the end of a line, which is defined as either the end of the string, or any location followed by a newline character.

5. \A Matches only at the start of the string. When not in MULTILINE mode, \A and ^ are effectively the same. In MULTILINE mode, they’re different: \A still matches only at the beginning of the string, but ^ may match at any location inside the string that follows a newline character.

6. \Z Matches only at the end of the string.

7. \b Word boundary. This is a zero-width assertion that matches only at the beginning or end of a word. A word is defined as a sequence of alphanumeric characters, so the end of a word is indicated by whitespace or a non-alphanumeric character.

8. \B Another zero-width assertion, this is the opposite of \b, only matching when the current position is not at a word boundary.

#### Example - Meta Character `.` 

In [181]:
# Any character except new line
test_string="a9."
pattern = re.compile(r".")
matches = pattern.finditer(test_string)

for match in matches:
    print(match.group())

a
9
.


Dot is a special character for any character. Hence by using meta character `\` convert it as raw searchable character

In [182]:
test_string="a9."
pattern = re.compile(r"\.")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(2, 3), match='.'>


#### Example - Meta Character `^` - Start With

In [60]:
test_string = '123abc456789abc123ABC.'
pattern = re.compile(r"^123")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 3), match='123'>


In [192]:
# Returns nothing when used with out multiline compilation flag
# It looks only at the begining of the line
test_string = '''
123
abc456789
abc123
ABC.'''
pattern = re.compile(r"^abc\w+")
matches = pattern.findall(test_string)
matches

[]

In [194]:
# Returns data when used with multiline compilation flag
test_string = '''
123
abc456789
abc123
ABC.'''
pattern = re.compile(r"^abc\w+", re.M)
matches = pattern.findall(test_string)
matches

['abc456789', 'abc123']

#### Example - Meta Character `$` - End With

In [203]:
p = re.compile(r'class$')

print(p.search('no class at all'))
print(p.search('one subclass'))

None
<re.Match object; span=(7, 12), match='class'>


In [199]:
test_string = '123abc456789abc123ABC.'
pattern = re.compile(r"\.$")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(21, 22), match='.'>


In [205]:
test_string = 'hello 123_ heyho hohey'
pattern = re.compile(r"\d")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(6, 7), match='1'>
<re.Match object; span=(7, 8), match='2'>
<re.Match object; span=(8, 9), match='3'>


In [207]:
test_string = 'hello 123_'
pattern = re.compile(r"\D")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 1), match='h'>
<re.Match object; span=(1, 2), match='e'>
<re.Match object; span=(2, 3), match='l'>
<re.Match object; span=(3, 4), match='l'>
<re.Match object; span=(4, 5), match='o'>
<re.Match object; span=(5, 6), match=' '>
<re.Match object; span=(9, 10), match='_'>


In [210]:
test_string = 'hello 123_'
pattern = re.compile(r"\s")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(5, 6), match=' '>


In [211]:
test_string = 'hello 123_'
pattern = re.compile(r"\S")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 1), match='h'>
<re.Match object; span=(1, 2), match='e'>
<re.Match object; span=(2, 3), match='l'>
<re.Match object; span=(3, 4), match='l'>
<re.Match object; span=(4, 5), match='o'>
<re.Match object; span=(6, 7), match='1'>
<re.Match object; span=(7, 8), match='2'>
<re.Match object; span=(8, 9), match='3'>
<re.Match object; span=(9, 10), match='_'>


In [214]:
test_string = 'hello 123_'
pattern = re.compile(r"\w")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 1), match='h'>
<re.Match object; span=(1, 2), match='e'>
<re.Match object; span=(2, 3), match='l'>
<re.Match object; span=(3, 4), match='l'>
<re.Match object; span=(4, 5), match='o'>
<re.Match object; span=(6, 7), match='1'>
<re.Match object; span=(7, 8), match='2'>
<re.Match object; span=(8, 9), match='3'>
<re.Match object; span=(9, 10), match='_'>


In [216]:
test_string = 'hello 123_'
pattern = re.compile(r"\W")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(5, 6), match=' '>


In [220]:
test_string = 'hello 123_'
pattern = re.compile(r"\bh")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 1), match='h'>


In [222]:
test_string = 'hello 123_ heyho hohey'
pattern = re.compile(r"\Bhey")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(19, 22), match='hey'>


In [85]:
test_string ='helloHELLO 123_'
pattern = re.compile(r"[a-zA-Z0-9]")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 1), match='h'>
<re.Match object; span=(1, 2), match='e'>
<re.Match object; span=(2, 3), match='l'>
<re.Match object; span=(3, 4), match='l'>
<re.Match object; span=(4, 5), match='o'>
<re.Match object; span=(5, 6), match='H'>
<re.Match object; span=(6, 7), match='E'>
<re.Match object; span=(7, 8), match='L'>
<re.Match object; span=(8, 9), match='L'>
<re.Match object; span=(9, 10), match='O'>
<re.Match object; span=(11, 12), match='1'>
<re.Match object; span=(12, 13), match='2'>
<re.Match object; span=(13, 14), match='3'>


In [87]:
test_string ='hello 123_'
pattern = re.compile(r"\d*")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(1, 1), match=''>
<re.Match object; span=(2, 2), match=''>
<re.Match object; span=(3, 3), match=''>
<re.Match object; span=(4, 4), match=''>
<re.Match object; span=(5, 5), match=''>
<re.Match object; span=(6, 9), match='123'>
<re.Match object; span=(9, 9), match=''>
<re.Match object; span=(10, 10), match=''>


In [88]:
test_string ='hello 123_'
pattern = re.compile(r"\d+")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(6, 9), match='123'>


In [89]:
test_string ='hello_123'
pattern = re.compile(r"_\d")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(5, 7), match='_1'>


In [90]:
test_string ='hello123'
pattern = re.compile(r"_\d")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

In [91]:
test_string ='hello123'
pattern = re.compile(r"_?\d")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(5, 6), match='1'>
<re.Match object; span=(6, 7), match='2'>
<re.Match object; span=(7, 8), match='3'>


In [92]:
test_string ='hello123'
pattern = re.compile(r"\d{3}")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(5, 8), match='123'>


In [95]:
test_string ='hello123'
pattern = re.compile(r"\d{1,5}")
matches = pattern.finditer(test_string)

for match in matches:
    print(match)

<re.Match object; span=(5, 8), match='123'>


In [96]:
dates ="""
01.04.2020

2020.04.01


2020-04-01
2020-05-23
2020-06-11
2020-07-11
2020-08-11

2020/04/02

2020_04_04
2020_04_04
"""

In [102]:
pattern = re.compile(r"\d{4}-\d{2}-\d{2}")
matches = pattern.finditer(dates)

for match in matches:
    print(match)

<re.Match object; span=(26, 36), match='2020-04-01'>
<re.Match object; span=(37, 47), match='2020-05-23'>
<re.Match object; span=(48, 58), match='2020-06-11'>
<re.Match object; span=(59, 69), match='2020-07-11'>
<re.Match object; span=(70, 80), match='2020-08-11'>


In [108]:
pattern = re.compile(r"\d{4}[-_]0[5-7][-_]\d{2}")
matches = pattern.finditer(dates)

for match in matches:
    print(match)

<re.Match object; span=(37, 47), match='2020-05-23'>
<re.Match object; span=(48, 58), match='2020-06-11'>
<re.Match object; span=(59, 69), match='2020-07-11'>


### Conditions

In [12]:
my_string = """
hello world
1223
2020-05-20
Mr Simpson
Mrs Simpson
Mr. Brown
Ms Smith
Mr. T
Mrs. Sujaya"""

In [13]:
pattern = re.compile(r"Mr\s\w+")
matches = pattern.finditer(my_string)

for match in matches:
    print(match)

<re.Match object; span=(29, 39), match='Mr Simpson'>


In [14]:
pattern = re.compile(r"Mr[.]\s\w+")
matches = pattern.finditer(my_string)

for match in matches:
    print(match)

<re.Match object; span=(52, 61), match='Mr. Brown'>
<re.Match object; span=(71, 76), match='Mr. T'>


In [15]:
pattern = re.compile(r"(Mr|Ms|Mrs)[.]?\s\w+")
matches = pattern.finditer(my_string)

for match in matches:
    print(match)

<re.Match object; span=(29, 39), match='Mr Simpson'>
<re.Match object; span=(40, 51), match='Mrs Simpson'>
<re.Match object; span=(52, 61), match='Mr. Brown'>
<re.Match object; span=(62, 70), match='Ms Smith'>
<re.Match object; span=(71, 76), match='Mr. T'>
<re.Match object; span=(77, 88), match='Mrs. Sujaya'>


In [16]:
pattern = re.compile(r"(Mr|Ms|Mrs)\.?\s\w+")
matches = pattern.finditer(my_string)

for match in matches:
    print(match)

<re.Match object; span=(29, 39), match='Mr Simpson'>
<re.Match object; span=(40, 51), match='Mrs Simpson'>
<re.Match object; span=(52, 61), match='Mr. Brown'>
<re.Match object; span=(62, 70), match='Ms Smith'>
<re.Match object; span=(71, 76), match='Mr. T'>
<re.Match object; span=(77, 88), match='Mrs. Sujaya'>


In [19]:
my_string = """
hello world
1223
2020-05-20
Mr Simpson
Mrs Simpson
Mr. Brown
Ms Smith
Mr. T
pythonengineer@gmail.com
python-engineer@gmx.de
python-engineer123@my-domain.org
"""

In [21]:
pattern = re.compile(r"[a-zA-Z0-9-]+@")
matches = pattern.finditer(my_string)

for match in matches:
    print(match)

<re.Match object; span=(77, 92), match='pythonengineer@'>
<re.Match object; span=(102, 118), match='python-engineer@'>
<re.Match object; span=(125, 144), match='python-engineer123@'>


In [20]:
pattern = re.compile(r"[\w-]+@")
matches = pattern.finditer(my_string)

for match in matches:
    print(match)

<re.Match object; span=(77, 92), match='pythonengineer@'>
<re.Match object; span=(102, 118), match='python-engineer@'>
<re.Match object; span=(125, 144), match='python-engineer123@'>


In [23]:
pattern = re.compile(r"[a-zA-Z0-9-]+@[a-zA-Z-]+\.[a-z]+")
matches = pattern.finditer(my_string)

for match in matches:
    print(match)

<re.Match object; span=(77, 101), match='pythonengineer@gmail.com'>
<re.Match object; span=(102, 124), match='python-engineer@gmx.de'>
<re.Match object; span=(125, 157), match='python-engineer123@my-domain.org'>


In [31]:
pattern = re.compile(r"""
                     [\w-]+    # Email Id 
                     @         # 
                     [\w-]+    # Domain Name
                     \.        # Dot
                     [\w]+     # Domain Extension
                     """, re.VERBOSE)

matches = pattern.finditer(my_string )

for match in matches:
    print(match)

<re.Match object; span=(77, 101), match='pythonengineer@gmail.com'>
<re.Match object; span=(102, 124), match='python-engineer@gmx.de'>
<re.Match object; span=(125, 157), match='python-engineer123@my-domain.org'>


### Grouping

Frequently you need to obtain more information than just whether the RE matched or not. Regular expressions are often used to dissect strings by writing a RE divided into several subgroups which match different components of interest.

Groups are marked by the '(', ')' metacharacters. '(' and ')' have much the same meaning as they do in mathematical expressions; they group together the expressions contained inside them, and you can repeat the contents of a group with a repeating qualifier, such as `*`, `+`, `?`, or `{m,n}`. For example, `(ab)*` will match zero or more repetitions of `ab`.

Groups indicated with '(', ')' also capture the starting and ending index of the text that they match; this can be retrieved by passing an argument to `group()`, `start()`, `end()`, and `span()`. Groups are numbered starting with 0. Group 0 is always present; it’s the whole RE, so match object methods all have group 0 as their default argument.

Subgroups are numbered from left to right, from 1 upward. Groups can be nested; to determine the number, just count the opening parenthesis characters, going from left to right.



In [227]:
pattern = re.compile(r"([a-zA-Z0-9-]+)@([a-zA-Z-]+)\.([a-z]+)")
matches = pattern.finditer(my_string)

for match in matches:
    print(match.group())
    print(match.group(0))
    print(match.group(1))
    print(match.group(2))
    print(match.group(3))

pythonengineer@gmail.com
pythonengineer@gmail.com
pythonengineer
gmail
com
python-engineer@gmx.de
python-engineer@gmx.de
python-engineer
gmx
de
python-engineer123@my-domain.org
python-engineer123@my-domain.org
python-engineer123
my-domain
org


`group()` can be passed multiple group numbers at a time, in which case it will return a tuple containing the corresponding values for those groups.

In [228]:
pattern = re.compile(r"([a-zA-Z0-9-]+)@([a-zA-Z-]+)\.([a-z]+)")
matches = pattern.finditer(my_string)

for match in matches:
    print(match.group(1, 2, 3))

('pythonengineer', 'gmail', 'com')
('python-engineer', 'gmx', 'de')
('python-engineer123', 'my-domain', 'org')


The `groups()` method returns a tuple containing the strings for all the subgroups, from 1 up to however many there are.

In [229]:
pattern = re.compile(r"([a-zA-Z0-9-]+)@([a-zA-Z-]+)\.([a-z]+)")
matches = pattern.finditer(my_string)

for match in matches:
    print(match.groups())

('pythonengineer', 'gmail', 'com')
('python-engineer', 'gmx', 'de')
('python-engineer123', 'my-domain', 'org')


Backreferences in a pattern allow you to specify that the contents of an earlier capturing group must also be found at the current location in the string. For example, \1 will succeed if the exact contents of group 1 can be found at the current position, and fails otherwise. Remember that Python’s string literals also use a backslash followed by numbers to allow including arbitrary characters in a string, so be sure to use a raw string when incorporating backreferences in a RE.

For example, the following RE detects doubled words in a string.

In [230]:
p = re.compile(r'\b(\w+)\s+\1\b')
p.search('Paris in the the spring').group()

'the the'

### Modifications

Split and Sub

In [232]:
p = re.compile(r'\W+')
p.split('This is a test, short and sweet, of split().')

['This', 'is', 'a', 'test, short and sweet, of split().']

In [233]:
p.split('This is a test, short and sweet, of split().', 3)

['This', 'is', 'a', 'test, short and sweet, of split().']

In [140]:
test_string = '123abc456abc123ABC' 

pattern = re.compile(r"abc")
splitted = pattern.split(test_string)
print(splitted)

['123', '456', '123ABC']


#### Search and Replace

Another common task is to find all the matches for a pattern, and replace them with a different string. The sub() method takes a replacement value, which can be either a string or a function, and the string to be processed.

```python
.sub(replacement, string[, count=0])
```

Returns the string obtained by replacing the leftmost non-overlapping occurrences of the RE in string by the replacement replacement. If the pattern isn’t found, string is returned unchanged.

The optional argument count is the maximum number of pattern occurrences to be replaced; count must be a non-negative integer. The default value of 0 means to replace all occurrences.

Here’s a simple example of using the sub() method. It replaces colour names with the word colour:

In [234]:
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes')

'colour socks and colour shoes'

In [235]:
p.sub('colour', 'blue socks and red shoes', count=1)

'colour socks and red shoes'

The subn() method does the same work, but returns a 2-tuple containing the new string value and the number of replacements that were performed:

In [236]:
p = re.compile('(blue|white|red)')
p.subn('colour', 'blue socks and red shoes')

('colour socks and colour shoes', 2)

In [237]:
p.subn('colour', 'no colours at all')

('no colours at all', 0)

In [145]:
test_string = 'hello world, you are the best world' 

pattern = re.compile(r"world")
subbed_string = pattern.sub("planet", test_string)
print(subbed_string)


hello planet, you are the best planet


In [146]:
test_string = 'hello world, you are the best world'
print(len(test_string))
pattern = re.compile(r",")
subbed_string = pattern.sub("", test_string)
print(subbed_string)
print(len(subbed_string))

35
hello world you are the best world
34


In [152]:
urls = '''
hello
2020-05-20
http://python-engineer.com
https://www.python-engineer.com
http://www.pyeng.net
'''

In [166]:
pattern = re.compile(r"https?://(www\.)?([a-zA-Z-]+)\.([a-zA-Z]+)")
matches = pattern.finditer(urls)

for match in matches:
    print(match)

<re.Match object; span=(18, 44), match='http://python-engineer.com'>
<re.Match object; span=(45, 76), match='https://www.python-engineer.com'>
<re.Match object; span=(77, 97), match='http://www.pyeng.net'>


In [184]:
urls = '''
http://python-engineer.com
https://www.python-engineer.com
http://www.pyeng.net
'''
pattern = re.compile(r"https?://(www\.)?([a-zA-Z-]+)(\.[a-zA-Z]+)")
matches = pattern.finditer(urls)

for match in matches:
    print(match)
    
subbed_urls = pattern.sub(r"\2\3", urls)
print(subbed_urls)
pattern = re.compile(r"[a-zA-Z-]+\.[a-z]+")
matches = pattern.findall(subbed_urls)
for match in matches:
    print(match)

<re.Match object; span=(1, 27), match='http://python-engineer.com'>
<re.Match object; span=(28, 59), match='https://www.python-engineer.com'>
<re.Match object; span=(60, 80), match='http://www.pyeng.net'>

python-engineer.com
python-engineer.com
pyeng.net

python-engineer.com
python-engineer.com
pyeng.net


In [187]:
my_string ="My Name Is Saravanan Shanmugham"
pattern = re.compile(r"[a-zA-Z]+")
matches = pattern.findall(my_string)
matches

['My', 'Name', 'Is', 'Saravanan', 'Shanmugham']

In [189]:
my_string ="My Name Is Saravanan Shanmugham"
pattern = re.compile(r"\S+")
matches = pattern.findall(my_string)
matches

['My', 'Name', 'Is', 'Saravanan', 'Shanmugham']

In [190]:
my_string ="My Name Is Saravanan Shanmugham"
pattern = re.compile(r"\w+")
matches = pattern.findall(my_string)
matches

['My', 'Name', 'Is', 'Saravanan', 'Shanmugham']

***

In [18]:
pattern = re.compile("o[gh]")
pattern.fullmatch("dog")      # No match as "o" is not at the start of "dog".
pattern.fullmatch("ogre")     # No match as not the full string matches.
pattern.fullmatch("doggie", 1, 3)   # Matches within given limits.


<re.Match object; span=(1, 3), match='og'>

In [30]:
valid = re.compile(r"^[abcde]{2}$")

print(valid.match("ab"))  # Valid.
print(valid.match("ec"))  # Valid.
print(valid.match("bd"))  # Valid.
print(valid.match("abhgde"))  # Invalid.
print(valid.match("a8"))  # Valid.



<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 2), match='ec'>
<re.Match object; span=(0, 2), match='bd'>
None
None


#### Making a Phonebook

split() splits a string into a list delimited by the passed pattern. The method is invaluable for converting textual data into data structures that can be easily read and modified by Python as demonstrated in the following example that creates a phonebook.

First, here is the input. Normally it may come from a file, here we are using triple-quoted string syntax

##### Documentation Reference:
```python
re.split(pattern, string, maxsplit=0, flags=0)
```

Split string by the occurrences of pattern. If capturing parentheses are used in pattern, then the text of all groups in the pattern are also returned as part of the resulting list. If maxsplit is nonzero, at most maxsplit splits occur, and the remainder of the string is returned as the final element of the list.

In [38]:
text = """Ross McFluff: 834.345.1254 155 Elm Street

Ronald Heathmore: 892.345.3428 436 Finley Avenue
Frank Burger: 925.541.7625 662 South Dogwood Way


Heather Albrecht: 548.326.4584 919 Park Place"""

The entries are separated by one or more newlines. Now we convert the string into a list with each nonempty line having its own entry:

In [39]:
entries = re.split("\n+", text)
entries


['Ross McFluff: 834.345.1254 155 Elm Street',
 'Ronald Heathmore: 892.345.3428 436 Finley Avenue',
 'Frank Burger: 925.541.7625 662 South Dogwood Way',
 'Heather Albrecht: 548.326.4584 919 Park Place']

Finally, split each entry into a list with first name, last name, telephone number, and address. We use the maxsplit parameter of split() because the address has spaces, our splitting pattern, in it:

In [40]:
[re.split(":? ", entry, 3) for entry in entries]

[['Ross', 'McFluff', '834.345.1254', '155 Elm Street'],
 ['Ronald', 'Heathmore', '892.345.3428', '436 Finley Avenue'],
 ['Frank', 'Burger', '925.541.7625', '662 South Dogwood Way'],
 ['Heather', 'Albrecht', '548.326.4584', '919 Park Place']]

The :? pattern matches the colon after the last name, so that it does not occur in the result list. With a maxsplit of 4, we could separate the house number from the street name:

In [46]:
[re.split(":? ", entry, 4) for entry in entries]

[['Ross', 'McFluff', '834.345.1254', '155', 'Elm Street'],
 ['Ronald', 'Heathmore', '892.345.3428', '436', 'Finley Avenue'],
 ['Frank', 'Burger', '925.541.7625', '662', 'South Dogwood Way'],
 ['Heather', 'Albrecht', '548.326.4584', '919', 'Park Place']]

### Text Munging

sub() replaces every occurrence of a pattern with a string or the result of a function. This example demonstrates using sub() with a function to “munge” text, or randomize the order of all the characters in each word of a sentence except for the first and last characters:


##### Documentation Reference
```python
re.sub(pattern, repl, string, count=0, flags=0)
```
Return the string obtained by replacing the leftmost non-overlapping occurrences of pattern in string by the replacement repl. If the pattern isn’t found, string is returned unchanged. **repl can be a string or a function**.

If it is a string, any backslash escapes in it are processed. That is, `\n` is converted to a single newline character, `\r` is converted to a carriage return, and so forth. Unknown escapes of ASCII letters are reserved for future use and treated as errors. Other unknown escapes such as `\&` are left alone. Backreferences, such as `\6`, are replaced with the substring matched by group 6 in the pattern.

In [50]:
def repl(m):
    inner_word = list(m.group(2))
    random.shuffle(inner_word)
    return m.group(1) + "".join(inner_word) + m.group(3)

text = "Professor Abdolmalek, please report your absences promptly."

re.sub(r"(\w)(\w+)(\w)", repl, text)


'Pofeossrr Aldaoelbmk, plaese roeprt your aecebsns pmtrploy.'

In [56]:
my_string ="My Name Is Saravanan Shanmugham"
pattern = re.compile(r"(\w)(\w+)(\w)")
re.sub(r"(\w)(\w+)(\w)", repl, my_string)


'My Nmae Is Svaaranan Sagahhmunm'

In [70]:
# Find all words ending with "ly" and "ed"

text = "He was carefully disguised but captured quickly by police."
re.findall(r"(\w+ed|\w+ly)\b", text)

['carefully', 'disguised', 'captured', 'quickly']

***